We first need to transform requested_documents as a consolidated hashmap. We consolidate by the last name. We also store the key just with the Name of the person.

This means that people with same last name are stored as one key and their files len are added togethers

In [324]:
import json

REQUESTED_DOCUMENTS_PATH = '../requested_documents.json'

with open(REQUESTED_DOCUMENTS_PATH, 'r') as f:
    requested_documents = json.load(f)

full_name_from_reqsdocs = list(requested_documents.keys())

consolidated_requested_documents = {}

for k,v in requested_documents.items():
    last_name = k.split(',')[0]
    if last_name not in consolidated_requested_documents:
        consolidated_requested_documents[last_name] = 0
    for doc in v:
        if "annual" in doc or "transaction" in doc:
            consolidated_requested_documents[last_name] += 1

print(sum(consolidated_requested_documents.values()))
print(len(consolidated_requested_documents))

6474
627


Now the Student's RA_Collection is being converted into a nested hashmap.

* The first key is the page_number, the second key is a dict that contains all the USOGE_last_name that this student captured
* We have consolidated all usoge people by their last_names
* We also remove duplicate last_names by removing the ones with a lower pdf files count

In [325]:
import os

def build_directory_tree(root_path):
    tree = {}
    
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        
        if os.path.isdir(item_path):
            tree[item] = build_directory_tree(item_path)
        elif item.endswith('.pdf'):
            if 'files' not in tree:
                tree['files'] = []
            tree['files'].append(item)
    
    if 'files' in tree and len(tree) == 1:
        return tree['files']
    elif 'files' in tree:
        files_list = tree.pop('files')
        tree['files'] = files_list
    
    return tree

def _consolidate_ra_files(ra_files):
    ra_files = {}
    for k,v in ra_collection.items():
        if isinstance(v, dict):
            ra_files = ra_files | v
    return ra_files

def _clean_ra_files(ra_files):
    for page,dirs in ra_files.items():
        for k,v in dirs.items():
            if k[0] != "R":
                k = page + " - " + k

            cleaned_ra_files[page] = cleaned_ra_files.get(page, {})
            cleaned_ra_files[page][k] = v

    return cleaned_ra_files

def _remove_non_annual_or_transaction_files(ra_files):
    for page,dirs in ra_files.items():
        for k,v in dirs.items():
            if isinstance(v, dict):
                #change this to an empty list
                ra_files[page][k] = []
                continue
        
            to_remove = []
            for doc in v:
                if "278" not in doc:
                    to_remove.append(doc)
            for doc in to_remove:
                v.remove(doc)
    return ra_files

def _consolidate_ra_files_by_last_name(ra_files):
    consolidated_ra_files = {}
    for k,v in ra_files.items():
        consolidated_ra_files[k] = {}
        for k2,v2 in v.items():
            last_name = k2.split("-")[-1]
            last_name = last_name.split(",")[0].lower()
            last_name = last_name.strip()

            if last_name not in consolidated_ra_files[k]:
                consolidated_ra_files[k][last_name] = []

            consolidated_ra_files[k][last_name] = consolidated_ra_files[k][last_name] + v2

    return consolidated_ra_files

def _remove_dups_usoge_last_name(ra_files):
    from collections import defaultdict
    
    seen = set()
    check_usoge_last_name = []
    ra_files_to_remove = defaultdict(list)

    #first populate the check_usoge_last_name with the repeating last names
    for k,v in ra_files.items():
        for k2,v2 in v.items():
            if k2 in seen:
                check_usoge_last_name.append(k2)
            else:
                seen.add(k2)

    #now gather the k1 and k2 for the repeating last names
    for k,v in ra_files.items():
        for k2,v2 in v.items():
            if k2 in check_usoge_last_name:
                ra_files_to_remove[k2].append((k,k2))

    #now check which one have a larger file count
    for k,v in ra_files_to_remove.items():
        max_file_count = [v[0][0], v[0][1]]
        if len(v) > 1:
            #now check which one have a larger file count
            for ra_name, usoge_last_name in v:
                if len(ra_files[ra_name][usoge_last_name]) > len(ra_files[max_file_count[0]][max_file_count[1]]):
                    max_file_count = [ra_name, usoge_last_name]
            
            
            #now remove the one with the smaller file count
            for ra_name, usoge_last_name in v:
                print("\t", ra_name)
                if ra_name != max_file_count[0]:
                    #delete this entry
                    del ra_files[ra_name][usoge_last_name]

    return ra_files

RA_COLLECTION_PATH = '../RA_Collection'
ra_collection_tree = {os.path.basename(RA_COLLECTION_PATH): build_directory_tree(RA_COLLECTION_PATH)}
ra_collection = ra_collection_tree[os.path.basename(RA_COLLECTION_PATH)]
ra_files = _consolidate_ra_files(ra_collection)

cleaned_ra_files = {}

cleaned_ra_files = _clean_ra_files(ra_files)
cleaned_ra_files = _remove_non_annual_or_transaction_files(cleaned_ra_files)
ra_files_full_name = cleaned_ra_files
ra_files = _consolidate_ra_files_by_last_name(ra_files_full_name)
ra_files = _remove_dups_usoge_last_name(ra_files)

#storing this as a json file with proper formatting
RA_COLLECTION_TREE_PATH = './JSON_FILES/ra_collection_tree.json'

with open(RA_COLLECTION_TREE_PATH, 'w') as f:
    json.dump(ra_files, f, indent=4)

	 RA collection - Ha Man Yan - Page 40
	 RA_Collection - Aarav_Kumar - Page_39
	 RA Collection - Lai Chun Kiu - Page 20
	 RA Collection - Lai Chun Kiu - Page 19
	 RA Collection - Lai Chun Kiu - Page 19
	 RA collection - Leong Deng Hung - Page 18
	 RA collection - JIAO DI - Page 50
	 RA Collection - ZHANG Xuanrui - Page 51
	 RA_Collection - Aarav_Kumar - Page_38
	 RA_Collection - Aarav_Kumar - Page_37
	 RA_Collection - Aarav_Kumar - Page_37
	 RA collection - Cayenne Tan - Page 36
	 RA collection - LI Junhui - Page 15
	 RA collection - LI Junhui - Page 14
	 RA collection - LI Junhui - Page 15
	 RA collection - Leong Deng Hung - Page 16
	 RA collection - LI Junhui - Page 14
	 RA collection - LI Junhui - Page 13
	 RA collection - LI Junhui - Page 53
	 RA collection - Leong Deng Hung - Page 54
	 RA collection - LI Junhui - Page 53
	 RA collection - Leong Deng Hung - Page 54
	 RA collection - Cayenne Tan - Page 34
	 RA collection - SUNG, Wen-chien - Page 33
	 RA Collection - LIU, Xucheng - P

How many total people (USOGE) did the students capture. How many of these are repeated across different students.

In [326]:
def _get_total_people_and_dups(ra_files):
    total_people = 0
    total_people_list = []
    seen_people = set()
    dups = 0

    for page,dirs in ra_files.items():
        for k,v in dirs.items():
            person = k

            if person not in seen_people:
                seen_people.add(person)
                total_people_list.append((person, k))
            else:
                dups += 1

            total_people += 1

    return total_people, dups, total_people_list

total_people, dups, total_people_list = _get_total_people_and_dups(ra_files)
print(total_people, dups)
print(len(total_people_list))

ppl_in_ra = set()
for ppl in total_people_list:
    ppl_in_ra.add(ppl[0])

print(len(ppl_in_ra))

677 0
677
677


How many of the people from requested_documents are not in RA collection. All based on last name [.split(",")[0]]

In [327]:
#gives the name of USOGE people who are not in requested_documents.json
def _get_missing_from_reqsdocs(ra_files, consolidated_requested_documents):

    missing_from_reqsdocs = []

    for page,dirs in ra_files.items():
        for k,v in dirs.items():
            person = k.split('-')[-1]
            person = person.split(',')[0].lower()
            person = person.strip()
            
            if person not in consolidated_requested_documents:
                missing_from_reqsdocs.append((person, k))

    return missing_from_reqsdocs

#gives the name of the USOGE people who are not in the RA collection
def _get_missing_from_ra(ppl_in_ra, consolidated_requested_documents):

    missing_from_ra = []

    for k,v in consolidated_requested_documents.items():
        if k not in ppl_in_ra:
            missing_from_ra.append(k)

    return missing_from_ra

missing_from_reqsdocs = _get_missing_from_reqsdocs(ra_files_full_name, consolidated_requested_documents)
print("People missing from reqsdocs: ", len(missing_from_reqsdocs))

missing_from_ra = _get_missing_from_ra(ppl_in_ra, consolidated_requested_documents)
print("People missing from ra: ", len(missing_from_ra))

print("please check the results after checking again from below")

print(missing_from_reqsdocs)

People missing from reqsdocs:  116
People missing from ra:  62
please check the results after checking again from below
[('raimondo', 'RA collection - Ha Man Yan - Page 40 - Raimondo, Gina M'), ('quarles', 'RA collection - Ha Man Yan - Page 40 - Quarles, Randal K'), ('remy', 'RA collection - Ha Man Yan - Page 41 - Remy, Donald'), ('rouse', 'RA collection - Ha Man Yan - Page 42 - Rouse, Cecilia'), ('ross', 'RA collection - Ha Man Yan - Page 42 - Ross, Wilbur L'), ('hegseth', 'RA Collection - Lai Chun Kiu - Page 20 - Hegseth, Pete'), ('niland', 'RA Collection - Lai Chun Kiu - Page 20 - Hennessey-Niland, John T'), ('hamilton', 'RA Collection - Lai Chun Kiu - Page 19 - Hamilton, Joseph B'), ('haaland', 'RA Collection - Lai Chun Kiu - Page 19 - Haaland, Debra A'), ('saric', 'RA Collection - Lai Chun Kiu - Page 21 - Hoehn-Saric, Alex'), ('burgum', 'RA collection -  JIAO DI - Page 6 - Burgum, Douglas J'), ('brouillette', 'RA collection - JIAO DI - Page 5 - Brouillette, Danny R'), ('bessent', 

In [328]:
def _get_missing_files_for_every_ra(ra_files, consolidated_requested_documents):

    missing_pages = {}

    for page,dirs in ra_files.items():
        missing_pages[page] = {}
        for k,v in dirs.items():
            person = k.split(' - ')[-1]
            person = person.split(',')[0].lower()

            #clean up the values. Only count the ones that have 278 in their name
            in_ra = 0
            seen = set()
            for docs in v:
                if docs not in seen:
                    if "278" in docs:
                        in_ra += 1
                    seen.add(docs)

            if person in consolidated_requested_documents:
                in_reqs = consolidated_requested_documents[person]
                missing_pages[page][k] = in_reqs - in_ra

    missing_files_from_ra = {}
    
    for ra, usoge_name in missing_pages.items():
        missing_files_from_ra[ra] = {}
        for k,v in usoge_name.items():
            missing_files_from_ra[ra][k] = v
    
    
    return missing_files_from_ra

def _check_again_for_those_in_ra_but_not_in_reqsdocs(ra_files, missing_from_reqsdocs, full_name_from_reqsdocs, final_res, requested_documents):

    match, not_match = [], []

    for last_name, dir in missing_from_reqsdocs:

        #get the whole name
        whole_name = dir.split("-")[-1]
        whole_name = whole_name.strip().lower()
        whole_name = whole_name.replace("_", " ")

        not_match.append(whole_name)
        for full_name in full_name_from_reqsdocs:
            if whole_name in full_name:
                match.append((whole_name, full_name, dir))
                not_match.pop()
                break

    cleaned_match = []

    for whole_name, full_name, dir in match:
        #extract the ra_name from the dir
        for k,v in ra_files_full_name.items():
            for k2,v2 in v.items():
                if k2 == dir:
                    #extract the ra_name from the k
                    ra_name = k
                    #get last name only
                    whole_name = whole_name.split(",")[0]
                    whole_name = whole_name.strip().lower()
                    cleaned_match.append((whole_name, full_name, ra_name))

    for last_name, reqs_name, ra_page in cleaned_match:
        if last_name in ra_files[ra_page]:
            #found the guy
            ra_files_count = len(ra_files[ra_page][last_name])
            final_reqs_docs = len(requested_documents[reqs_name])

            result = final_reqs_docs - ra_files_count
            #now insert this into the final_res
            final_res[ra_page][last_name] = result

    return final_res

def _add_count_positive_vals(missing_files_from_ra):
    
    missing_files_count = 0

    for k,v in missing_files_from_ra.items():
        total_missing_files_count = 0

        for k2,v2 in v.items():
            if v2 > 0:
                total_missing_files_count += v2

        missing_files_from_ra[k]['Total Number of Files Missing (considering only positive values)'] = total_missing_files_count
        missing_files_count += total_missing_files_count

    return missing_files_from_ra, missing_files_count


missing_files_from_ra = _get_missing_files_for_every_ra(ra_files, consolidated_requested_documents)
missing_files_from_ra = _check_again_for_those_in_ra_but_not_in_reqsdocs(ra_files, missing_from_reqsdocs, full_name_from_reqsdocs, missing_files_from_ra, requested_documents)
missing_files_from_ra, total_missing_files_count = _add_count_positive_vals(missing_files_from_ra)

MISSING_FILES_SUMMARY_PATH = './JSON_FILES/missing_files_for_every_ra.json'
with open(MISSING_FILES_SUMMARY_PATH, 'w') as f:
    json.dump(missing_files_from_ra, f, indent=4)

print(total_missing_files_count)

797


Hunch: Students did not consolidate USOGE people based on the last name. Lets try to do that. Maybe the script is double counting

In [329]:
def _get_who_did_not_consolidate_ra_people(missing_files_from_ra, consolidated_requested_documents):
    seen = set()
    not_consolidated = []
    for k,v in missing_files_from_ra.items():
        for k2,v2 in v.items():
            if k2[0] == "T":
                continue

            last_name = k2.split(" - ")[-1]
            last_name = last_name.split(",")[0].lower()
            last_name = last_name.strip()

            if last_name not in seen:
                seen.add(last_name)
            else:
                not_consolidated.append(k2)

    return not_consolidated

not_consolidated = _get_who_did_not_consolidate_ra_people(missing_files_from_ra, consolidated_requested_documents)
print(len(not_consolidated))
for name in not_consolidated:
    print(name)

0
